### Load preprocessed data

In [13]:
import numpy as np
fh = np.load('../data/dataset.npz')
# We have a bunch of feature columns and last column is the y-target
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']

n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

### Define the MF Model

In [52]:
import torch
from torch import nn
import torch.nn.functional as F

def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


class MF(nn.Module):
    itr = 0
    
    def __init__(self, n_user, n_item, k=18, c=4,
                 c_vector=1.0, c_bias=1.0, writer=None):
        super(MF, self).__init__()
        self.writer = writer
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_bias = c_bias
        self.c_vector = c_vector
        
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
        self.item = nn.Embedding(n_item, k)
        
        # **NEW: user taste & attention vectors
        user_taste = torch.zeros(n_user, k, c)
        user_attnd = torch.zeros(n_user, k, c)
        user_taste.data.normal_(0, 1.0 / n_user)
        user_attnd.data.normal_(0, 1.0 / n_user)
        
        self.user_taste = nn.Parameter(user_taste)
        self.user_attnd = nn.Parameter(user_attnd)

    
    def __call__(self, train_x):
        user_id = train_x[:, 0]
        item_id = train_x[:, 1]
        vector_item = self.item(item_id)
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)
        
        # **NEW: user taste & attention
        user_taste = self.user_taste[user_id]
        user_attnd = self.user_attnd[user_id]
        vector_itemx = vector_item.unsqueeze(2).expand_as(user_attnd)
        attention = F.softmax(user_attnd * vector_itemx)
        attentionx = attention.sum(2).unsqueeze(2).expand_as(user_attnd)
        weighted_preference = (user_taste * attentionx).sum(2)
        dot = (weighted_preference * vector_item).sum(1)
        
        prediction = dot + biases
        return prediction
    
    def loss(self, prediction, target):
        loss_mse = F.mse_loss(prediction.squeeze(), target.squeeze())
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        prior_taste =  l2_regularize(self.user_taste) * self.c_vector
        prior_attnd =  l2_regularize(self.user_attnd) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        
        total = (loss_mse + prior_bias_item + prior_bias_user +
                 prior_taste + prior_attnd + prior_item)
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total

### Train model

In [53]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from tensorboardX import SummaryWriter
from ignite.metrics import MeanSquaredError

from loader import Loader

#### Hyperparameters

In [54]:
n_epochs = 10
k = 10
c = 4
c_bias = 1e-6
c_vector = 1e-6

In [55]:
writer = SummaryWriter()
model = MF(n_user, n_item,  k=k, c=c, c_bias=c_bias, 
           c_vector=c_vector, writer=writer)
optimizer = torch.optim.Adam(model.parameters())
trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)
train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)


def log_training_loss(engine, log_interval=400):
    epoch = engine.state.epoch
    itr = engine.state.iteration
    fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
    msg = fmt.format(epoch, itr, len(train_loader), engine.state.output)
    model.itr = itr
    if itr % log_interval == 0:
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    evaluat.run(test_loader)
    metrics = evaluat.state.metrics
    avg_accuracy = metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} "
          .format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MF(
  (bias_user): Embedding(6041, 1)
  (bias_item): Embedding(3953, 1)
  (item): Embedding(3953, 10)
)

#### Run model

In [ ]:
trainer.run(train_loader, max_epochs=50)